In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)   # validation의 정확도 최대일때 저장
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=5)
# tvalidation의 정확도 최대일때. patience는 기준되는 값이 연속으로 몇 번 이상 향상되지 않을 때 종료시킬 것인지

In [2]:
df = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,72,528,32,408,699
1,0,0,0,0,0,0,0,0,0,0,...,507,45,674,2,101,1360,36,1003,720,35
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,577,4,21,8,234,3
3,0,0,0,0,0,0,0,0,0,0,...,0,6396,125,5999,253,70,12,14,49,3566
4,0,0,0,0,0,0,0,0,0,0,...,1122,257,3,10090,11825,7,1116,1,1404,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155863,0,0,0,0,0,1087,4173,5,7939,8,...,15,40,73,182,39,444,66,10,266,596
155864,0,0,0,0,0,0,0,0,0,0,...,146,6,819,1907,659,15,3,692,4854,35
155865,0,0,0,0,0,0,0,0,0,0,...,105,191,105,191,95,45,61,1116,865,636
155866,0,0,0,0,0,0,0,0,0,0,...,5,285,1116,1610,41,5,1461,1116,93,3


In [3]:
X_train_data = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/okt/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/okt/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/okt/y_test.csv')

In [4]:
X_train_data.dropna(how='any', inplace=True)    # null값 탈락
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [5]:
train_X = np.array(X_train_data)    # array로 변경하여 tensor
test_X = np.array(X_test_data)
train_X, test_X

(array([[   0,    0,    0, ...,   32,  408,  699],
        [   0,    0,    0, ..., 1003,  720,   35],
        [   0,    0,    0, ...,    8,  234,    3],
        ...,
        [   0,    0,    0, ..., 1116,  865,  636],
        [   0,    0,    0, ..., 1116,   93,    3],
        [   0,    0,    0, ...,   95,  154,    4]], dtype=int64),
 array([[   0,    0,    0, ...,    0,  815,  119],
        [   0,    0,    0, ...,   64,  745,   34],
        [   0,    0,    0, ...,   86,  170,  295],
        ...,
        [   0,    0,    0, ...,  180, 3045,   40],
        [   0,    0,    0, ...,   11,  255,  151],
        [   0,    0,    0, ...,  143,   59,   46]], dtype=int64))

In [6]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building

In [7]:
vocab_size = 19395                                                   #전처리 이후 사이즈 확인

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))   # 임베딩 32
model.add(keras.layers.Dropout(0.5))                                 # dropout 0.2 - 0.5사이
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))          # 노드  64
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='softsign'))            # softsign, tanh로 돌려봄
model.add(keras.layers.Dropout(0.5))  
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          620640    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              37632     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                        

In [8]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)  # batch_size 64,128,256 확인
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [10]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc]) 
#callbacks로 체크포인트 확인 epochs 10,20회 반복으로 시도

Epoch 1/10
1218/1218 [==============================] - 80s 61ms/step - loss: 0.4218 - accuracy: 0.8080 - val_loss: 0.5001 - val_accuracy: 0.7488
Epoch 2/10
1218/1218 [==============================] - 76s 62ms/step - loss: 0.3565 - accuracy: 0.8517 - val_loss: 0.4320 - val_accuracy: 0.7949
Epoch 3/10
1218/1218 [==============================] - 78s 64ms/step - loss: 0.3388 - accuracy: 0.8620 - val_loss: 0.3942 - val_accuracy: 0.8234
Epoch 4/10
1218/1218 [==============================] - 75s 61ms/step - loss: 0.3217 - accuracy: 0.8702 - val_loss: 0.3829 - val_accuracy: 0.8260
Epoch 5/10
1218/1218 [==============================] - 110s 91ms/step - loss: 0.3093 - accuracy: 0.8767 - val_loss: 0.3809 - val_accuracy: 0.8265
Epoch 6/10
1218/1218 [==============================] - 122s 100ms/step - loss: 0.2974 - accuracy: 0.8817 - val_loss: 0.3640 - val_accuracy: 0.8356
Epoch 7/10
1218/1218 [==============================] - 115s 94ms/step - loss: 0.2854 - accuracy: 0.8877 - val_loss: 0.38

### 하이퍼 파라미터 조절
#### OKT, tanh 사용 epochs 10회씩
drop out ---> 0.2 
batch size --->   256 =>128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16


drop out --->    0.4
batch size ---> 128
layer ---->  32  , 중간 GRU(64)는 건들지 않음,  16


drop out --->  0.4 
batch size --->  128
layer ---->  32 =>16 , 중간 GRU(64)는 건들지 않음,  16=>8


drop out --->   0.5
batch size --->   128
layer ---->  16 , 중간 GRU(64)는 건들지 않음,  8


drop out --->   0.5
batch size --->   128
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   64
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16    =======> 최종 하이퍼파라미터

### 이후 
최종 하이퍼 파라미터 사용

OKT

softsign과 tanh 비교

### 최고 수치
# tanh         val_accuracy:0.8450,  val_loss:0.3586
# softsign   val_accuracy: 0.8474,  val_loss:0.3589